In [1]:
%matplotlib inline

In [36]:
from bw_recipe_2016 import *
import seaborn as sb
from matplotlib import pyplot as plt
import itertools
import numpy as np

In [5]:
from pathlib import Path

In [9]:
output_dir = Path("data").resolve()
output_dir.mkdir(exist_ok=True)

In [3]:
x = extract_recipe(0)
y = extract_recipe(1)
z = extract_recipe(2)

Skipping category Marine eutrophication, not found in /Users/cmutel/Code/bw-recipe-2016/bw_recipe_2016/data/ReCiPe2016_CFs_v1.0_20161004.xlsx
Skipping category Land transformation, not found in /Users/cmutel/Code/bw-recipe-2016/bw_recipe_2016/data/ReCiPe2016_CFs_v1.0_20161004.xlsx


In [26]:
terr = (x[10], y[11], z[11])
freshw = (x[11], y[12], z[12])
marine = (x[12], y[13], z[13])
carc = (x[13], y[14], z[14])
noncarc = (x[14], y[15], z[15])

In [31]:
perspectives = {key[1] for dct in itertools.chain(x[10:15], y[11:16], z[11:16]) for key in dct}
perspectives

{'E', 'Egalitarian', 'H', 'Hierarchic', 'I', 'Individualistic'}

In [33]:
def normalize_perspective(s):
    mapping = {
        'E': 'Egalitarian',
        'H': 'Hierarchic',
        'I': 'Individualistic',
    }
    return mapping.get(s, s)

In [20]:
categories = {cf['compartment'] 
              for obj in (x[10], x[11], x[12], x[13], x[14], y[11], y[15], z[11], z[15]) 
              for persp in obj.values() 
              for cf in persp['cfs']}
categories

{'Agricultural soil',
 'Freshwater',
 'Industrial soil',
 'Rural air',
 'Sea water',
 'Urban air',
 'agricultural soil',
 'freshwater',
 'industrial soil',
 'rural air',
 'sea water',
 'seawater',
 'urban  air',
 'urban air'}

In [21]:
def normalize_category(s):
    return s.lower().replace(" ", "")

In [23]:
normalized_categories = {normalize_category(s) for s in categories}
normalized_categories

{'agriculturalsoil',
 'freshwater',
 'industrialsoil',
 'ruralair',
 'seawater',
 'urbanair'}

In [47]:
def get_data_as_dict(compartment, perspective, data):
    data = [v for obj in data for k, v in obj.items() if normalize_perspective(k[1]) == perspective]
    assert len(data) == 3
    return [{cf['name']: cf['amount'] 
             for cf in obj['cfs'] if normalize_category(cf['compartment']) == compartment} 
            for obj in data]

In [78]:
def plot_ratio(compartment, perspective, data, filename, persp, comparison='first'):
    x, y, z = get_data_as_dict(compartment, perspective, data)
    
    plt.figure(figsize=(8, 6))
    
    if comparison == 'first':
        ratios = np.array([y[key] / x[key] for key in x if x[key] and y.get(key)])
        if max(ratios) < 1.1:
            print("Skipping", filename)
            plt.close()
            return
        sb.distplot(np.log10(ratios))
        plt.xlabel(r"$log_{10} \frac{1.1-20170929}{1.0-20161004}$")

    else:   
        ratios = np.array([z[key] / y[key] for key in y if y[key] and z.get(key)])
        if max(ratios) < 1.1:
            print("Skipping", filename)
            plt.close()
            return
        sb.distplot(np.log10(ratios))
        plt.xlabel(r"$log_{10} \frac{1.1-20180117}{1.1-20170929}$")

    plt.title(filename)
    plt.ylabel("Probability")

    (output_dir / persp).mkdir(exist_ok=True)
    plt.savefig(output_dir / persp / (filename + ".png"), dpi=300)
    plt.close()

In [79]:
datasets = [
    ("Terrestrial ecotox", terr),
    ("Freshwater ecotox", freshw),
    ("Marine ecotox", marine),
    ("Carcinogenic tox", carc),
    ("Non-carcinogenic tox", noncarc),
]

In [80]:
for persp in ('Egalitarian', 'Hierarchic', 'Individualistic'):
    for category in normalized_categories:
        for label, dataset in datasets:
            filename = "{} - {} - {} - 1.0".format(label, category, persp).title()
            plot_ratio(category, persp, dataset, filename, persp)
            
            filename = "{} - {} - {} - 1.1".format(label, category, persp).title()
            plot_ratio(category, persp, dataset, filename, persp, True)

Skipping Terrestrial Ecotox - Industrialsoil - Egalitarian - 1.1
Skipping Freshwater Ecotox - Industrialsoil - Egalitarian - 1.0
Skipping Freshwater Ecotox - Industrialsoil - Egalitarian - 1.1
Skipping Marine Ecotox - Industrialsoil - Egalitarian - 1.0
Skipping Marine Ecotox - Industrialsoil - Egalitarian - 1.1
Skipping Carcinogenic Tox - Industrialsoil - Egalitarian - 1.0
Skipping Carcinogenic Tox - Industrialsoil - Egalitarian - 1.1
Skipping Non-Carcinogenic Tox - Industrialsoil - Egalitarian - 1.1
Skipping Terrestrial Ecotox - Urbanair - Egalitarian - 1.1
Skipping Freshwater Ecotox - Urbanair - Egalitarian - 1.0
Skipping Freshwater Ecotox - Urbanair - Egalitarian - 1.1
Skipping Marine Ecotox - Urbanair - Egalitarian - 1.0
Skipping Marine Ecotox - Urbanair - Egalitarian - 1.1
Skipping Carcinogenic Tox - Urbanair - Egalitarian - 1.0
Skipping Carcinogenic Tox - Urbanair - Egalitarian - 1.1
Skipping Non-Carcinogenic Tox - Urbanair - Egalitarian - 1.1
Skipping Terrestrial Ecotox - Rurala

Skipping Non-Carcinogenic Tox - Freshwater - Individualistic - 1.1
Skipping Terrestrial Ecotox - Seawater - Individualistic - 1.1
Skipping Freshwater Ecotox - Seawater - Individualistic - 1.0
Skipping Freshwater Ecotox - Seawater - Individualistic - 1.1
Skipping Marine Ecotox - Seawater - Individualistic - 1.0
Skipping Marine Ecotox - Seawater - Individualistic - 1.1
Skipping Carcinogenic Tox - Seawater - Individualistic - 1.0
Skipping Carcinogenic Tox - Seawater - Individualistic - 1.1
Skipping Non-Carcinogenic Tox - Seawater - Individualistic - 1.1
